# 01 - Ingest and Clean: Core Sources

This notebook pulls and lightly cleans initial datasets for a small Gulf Coast subset to enable fast iteration.


In [1]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
import requests
from tqdm import tqdm

PROJECT_ROOT = Path('/Users/liamguest/LProjects/AURA/AURA')
DATA_RAW = PROJECT_ROOT / 'data' / 'raw'
DATA_INTERIM = PROJECT_ROOT / 'data' / 'interim'
DATA_RAW.mkdir(parents=True, exist_ok=True)
DATA_INTERIM.mkdir(parents=True, exist_ok=True)

print('Raw:', DATA_RAW)
print('Interim:', DATA_INTERIM)


Raw: /Users/liamguest/LProjects/AURA/AURA/data/raw
Interim: /Users/liamguest/LProjects/AURA/AURA/data/interim


## OpenFEMA Individual Assistance (IA) - Minimal Pull

Filters: LargeDisastersDataset


In [6]:
import time, requests, pandas as pd
from pathlib import Path

OPENFEMA_HOUSING_V1 = "https://www.fema.gov/api/open/v1/IndividualAssistanceHousingRegistrantsLargeDisasters"
RAW = Path("data/raw"); INT = Path("data/interim"); RAW.mkdir(parents=True, exist_ok=True); INT.mkdir(parents=True, exist_ok=True)

STATE_ABBR = ['TX','LA','MS','AL','FL']
states_str = "','".join(STATE_ABBR)

# Smaller column set
select_cols = ",".join([
  "id","disasterNumber","damagedStateAbbreviation","damagedZipCode","ownRent","residenceType",
  "grossIncome","specialNeeds","tsaEligible","repairAssistanceEligible","replacementAssistanceEligible",
  "personalPropertyEligible","ppfvl","censusBlockId","censusYear"
])

top = 5000           # reduce page size
max_pages = 5        # limit pages for a fast first pass (≈ up to 25k rows)
skip = 0
frames = []

for page in range(max_pages):
    params = {
        "$filter": f"damagedStateAbbreviation in ('{states_str}')",
        "$select": select_cols,
        "$format": "json",
        "$top": top,
        "$skip": skip
    }
    r = requests.get(OPENFEMA_HOUSING_V1, params=params, timeout=60)
    r.raise_for_status()
    data = r.json().get("IndividualAssistanceHousingRegistrantsLargeDisasters", [])
    if not data:
        break
    frames.append(pd.DataFrame(data))
    skip += top
    time.sleep(0.2)

fema = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
raw_out = RAW / "fema_housing_subset.json"
fema.to_json(raw_out, orient="records")
print("FEMA rows:", len(fema), "->", raw_out)

def to_tract_geoid(val):
    s = str(val) if pd.notna(val) else ""
    return s[:11] if len(s) >= 11 else None

if not fema.empty:
    fema["tract_geoid"] = fema["censusBlockId"].map(to_tract_geoid)
    clean_out = INT / "fema_housing_subset_clean.csv"
    fema.to_csv(clean_out, index=False)
    print("Wrote clean subset:", clean_out, "rows:", len(fema))

FEMA rows: 25000 -> data/raw/fema_housing_subset.json
Wrote clean subset: data/interim/fema_housing_subset_clean.csv rows: 25000


In [7]:
import pandas as pd
df = pd.read_csv("data/interim/fema_housing_subset_clean.csv")
print(df.shape)
df.head(5)

(25000, 16)


,id,disasterNumber,damagedStateAbbreviation,damagedZipCode,ownRent,residenceType,grossIncome,specialNeeds,tsaEligible,repairAssistanceEligible,replacementAssistanceEligible,personalPropertyEligible,ppfvl,censusBlockId,censusYear,tract_geoid
0,d214ed1e-951e-484b-b014-2a885a7ea234,4332,TX,77036,Renter,Apartment,1800.0,True,False,False,False,False,NaN,4.820143e+14,2010.0,4.820143e+10
1,e7b10c4f-adec-4a58-a4fc-85b5a20dfa4c,4337,FL,34238,Renter,Condo,NaN,False,False,False,False,False,NaN,1.211500e+14,2010.0,1.211500e+10
2,0832cc95-c445-4429-956c-e2d1864d37ac,4337,FL,34758,Renter,House/Duplex,28000.0,False,True,False,False,False,0.0,1.209704e+14,2010.0,1.209704e+10
3,ce6f1140-3777-41d8-8da0-f7f4891b2228,4559,LA,70663,Renter,Apartment,55000.0,False,False,False,False,False,NaN,2.201900e+14,2020.0,2.201900e+10
4,f47b5c56-74b6-4572-844b-64aaacb85234,4332,TX,77088,Renter,Apartment,23000.0,False,True,False,False,False,0.0,4.820153e+14,2010.0,4.820153e+10


## NOAA HURDAT2 - Best Track Data

We will download the Atlantic basin text file and parse into a tabular format.


In [ ]:
HURDAT_URL = 'https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2023-070924.txt'
hurdat_path = DATA_RAW / 'hurdat2_atlantic.txt'

r = requests.get(HURDAT_URL, timeout=60)
r.raise_for_status()
hurdat_path.write_bytes(r.content)
print('Wrote', hurdat_path)

# Quick parse sketch: collect header and record lines
lines = hurdat_path.read_text().splitlines()
records = []
current_storm = None
for line in lines:
    if line and line[0].isalpha():
        # Header: e.g., AL011851, UNNAMED, 14
        parts = [p.strip() for p in line.split(',')]
        current_storm = {'id': parts[0], 'name': parts[1], 'n': int(parts[2])}
    else:
        # Data line
        parts = [p.strip() for p in line.split(',')]
        if len(parts) >= 8 and current_storm:
            ymdh = parts[0]
            rec = {
                'storm_id': current_storm['id'],
                'storm_name': current_storm['name'],
                'date': ymdh[:8],
                'time': ymdh[8:],
                'record_id': parts[2],
                'status': parts[3],
                'lat': parts[4],
                'lon': parts[5],
                'max_wind_kt': parts[6],
                'min_pres_mb': parts[7]
            }
            records.append(rec)

hurdat_df = pd.DataFrame(records)
hurdat_csv = DATA_INTERIM / 'hurdat2_atlantic_parsed.csv'
hurdat_df.to_csv(hurdat_csv, index=False)
print('Parsed records:', len(hurdat_df), '->', hurdat_csv)


## ACS Demographics (Census API) - Tract Level

We will fetch a small set of variables for TX/LA tracts.


In [ ]:
CENSUS_BASE = 'https://api.census.gov/data/2022/acs/acs5'
# Example variables: total population, median household income
vars_ = ['NAME', 'B01003_001E', 'B19013_001E']
get = ','.join(['GEO_ID'] + vars_)

params = {
    'get': get,
    'for': 'tract:*',
}

acs_frames = []
for state in STATE_FIPS:
    p = params | {'in': f'state:{state}'}
    if CENSUS_API_KEY:
        p['key'] = CENSUS_API_KEY
    r = requests.get(CENSUS_BASE, params=p, timeout=60)
    r.raise_for_status()
    df = pd.DataFrame(r.json()[1:], columns=r.json()[0])
    df['state_fips'] = state
    acs_frames.append(df)

acs = pd.concat(acs_frames, ignore_index=True)
acs_out = DATA_INTERIM / 'acs_2022_tx_la_ms_al_fl.csv'
acs.to_csv(acs_out, index=False)
print('Wrote', acs_out, 'rows:', len(acs))


## Configuration: five Gulf states and API key

Defines state lists (TX, LA, MS, AL, FL) and reads your Census API key from the environment.


In [ ]:
import os

STATE_ABBR = ['TX', 'LA', 'MS', 'AL', 'FL']
STATE_FIPS = ['48', '22', '28', '01', '12']
CENSUS_API_KEY = os.getenv('CENSUS_API_KEY')
print('CENSUS_API_KEY set:', bool(CENSUS_API_KEY))


## CDC Social Vulnerability Index (SVI) - Placeholder

Download the latest SVI (tract-level) for TX, LA, MS, AL, FL from CDC/ATSDR. Place CSVs under `data/raw/svi/` and run the next cell to combine.


In [ ]:
svi_dir = DATA_RAW / 'svi'
svi_dir.mkdir(parents=True, exist_ok=True)

# Expect one or multiple CSVs dropped here; we will read all CSVs and concat
svi_frames = []
for p in sorted(svi_dir.glob('*.csv')):
    try:
        df = pd.read_csv(p, dtype=str)
        df['source_file'] = p.name
        svi_frames.append(df)
    except Exception as e:
        print('Failed to read', p, e)

if svi_frames:
    svi = pd.concat(svi_frames, ignore_index=True)
    svi_out = DATA_INTERIM / 'svi_combined.csv'
    svi.to_csv(svi_out, index=False)
    print('Wrote', svi_out, 'rows:', len(svi))
else:
    print('No SVI CSVs found in', svi_dir)


In [ ]:
import os, sys, platform
print(sys.executable, platform.python_version())
print('Has CENSUS_API_KEY:', bool(os.getenv('CENSUS_API_KEY')))

/Applications/anaconda3/envs/aura/bin/python 3.11.11
Has CENSUS_API_KEY: False
